## Replace Bellow cells 1 by 1 with domain specific knowlledge/information/data

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

# Reset random seed for 2025
np.random.seed(2025)

# =====================================================
# 1. COFFEE SERVICE TOUCHPOINTS (6 NODES)
# =====================================================
TOUCHPOINTS = [
    "Order_Intake",      # 0: POS entry & Queue
    "Brewing_Prep",      # 1: Extraction & Milk work
    "Quality_Audit",     # 2: Visual & Temp check
    "Payment_Finalize",  # 3: Transaction & Digital Tip
    "Handover_Vibe",     # 4: Customer interaction
    "Inventory_Log"      # 5: Waste/Stock deduction
]

# =====================================================
# 2. METRIC CONFIGURATION
# =====================================================
METRIC_KEYS = [
    "Throughput_Speed",    # 0
    "Product_Quality",     # 1
    "Customer_Experience", # 2
    "Operational_Margin"   # 3
]

METRIC_FEATURE_MAP = {
    "Order_Intake":     ['queue_wait_sec'],      # Speed
    "Brewing_Prep":     ['extraction_accuracy'], # Quality
    "Quality_Audit":    ['rejection_rate'],      # Quality/Experience
    "Payment_Finalize": ['transaction_speed'],   # Speed
    "Handover_Vibe":    ['loyalty_opt_in'],      # Experience
    "Inventory_Log":    ['waste_variance']       # Margin
}

# METRIC TARGET MATRIX (6x4)
METRIC_TARGET = np.array([
    [1, 0, 0, 0], # 0. Order -> Speed
    [0, 1, 0, 0], # 1. Prep -> Quality
    [0, 1, 1, 0], # 2. Audit -> Quality + Exp
    [1, 0, 0, 0], # 3. Payment -> Speed
    [0, 0, 1, 0], # 4. Handover -> Experience
    [0, 0, 0, 1]  # 5. Inventory -> Margin
])

# METRIC FORMULAS (Scales 0.0 - 1.0)
METRIC_FORMULAS = [
    lambda x: np.exp(-x / 300.0),       # 0. Order: Faster = better
    lambda x: x / 100.0,                # 1. Prep: Accuracy %
    lambda x: 1.0 - x,                  # 2. Audit: Low rejection = high score
    lambda x: 60 / (60 + x),            # 3. Payment: Speed normalization
    lambda x: 1.0 if x > 0.5 else 0.4,  # 4. Handover: Loyalty threshold
    lambda x: np.exp(-x / 10.0)         # 5. Inventory: Low waste = high margin
]

# =====================================================
# 3. COFFEE DSM (DEPENDENCY STRUCTURE)
# =====================================================
# Defines the flow and the "Production Knot" (Rework loop 1 <-> 2)
DSM = np.array([
    #0    1    2    3    4    5
    [0,   1,   0,   0,   0,   0],   # Order -> Prep
    [0,   0,   1,   0,   0,   0],   # Prep -> Audit
    [0,   0.4, 0,   1,   0,   0],   # Audit Feedback to Prep (Rework Loop)
    [0,   0,   0,   0,   1,   0],   # Payment -> Handover
    [0,   0,   0,   0,   0,   1],   # Handover -> Log
    [0,   0,   0,   0,   0,   0],   # Log (End)
])

# =====================================================
# 4. SIMULATOR CLASS
# =====================================================

class CoffeeSimulator:
    def __init__(self):
        self.generators = {
            "Order_Intake":     lambda: {"queue_wait_sec": np.random.exponential(120)},
            "Brewing_Prep":     lambda: {"extraction_accuracy": np.random.uniform(80, 100)},
            "Quality_Audit":    lambda: {"rejection_rate": np.random.beta(1, 10)},
            "Payment_Finalize": lambda: {"transaction_speed": np.random.uniform(5, 45)},
            "Handover_Vibe":    lambda: {"loyalty_opt_in": np.random.uniform(0, 1)},
            "Inventory_Log":    lambda: {"waste_variance": np.random.rayleigh(2)}
        }

    def run(self, dsm_matrix):
        G = nx.DiGraph()
        for i, u in enumerate(TOUCHPOINTS):
            for j, v in enumerate(TOUCHPOINTS):
                if dsm_matrix[i, j] > 0:
                    G.add_edge(u, v)

        # Cluster Detection (Strongly Connected Components)
        sccs = list(nx.strongly_connected_components(G))
        node_to_mod = {n: i for i, scc in enumerate(sccs) for n in scc}

        MG = nx.DiGraph()
        for u, v in G.edges():
            if node_to_mod[u] != node_to_mod[v]:
                MG.add_edge(node_to_mod[u], node_to_mod[v])

        levels = list(nx.topological_generations(MG))
        results = []

        for lvl in levels:
            for mod_idx in lvl:
                nodes = list(sccs[mod_idx])
                is_rework_cluster = len(nodes) > 1

                for node_name in nodes:
                    node_idx = TOUCHPOINTS.index(node_name)
                    data_dict = self.generators[node_name]()

                    primary_key = METRIC_FEATURE_MAP[node_name][0]
                    raw_val = data_dict[primary_key]
                    score = METRIC_FORMULAS[node_idx](raw_val)

                    for m_idx, active in enumerate(METRIC_TARGET[node_idx]):
                        if active:
                            results.append({
                                "Touchpoint": node_name,
                                "Feature": primary_key,
                                "Value": round(raw_val, 2),
                                "Score": round(score, 3),
                                "Metric": METRIC_KEYS[m_idx],
                                "Rework_Risk": is_rework_cluster
                            })

        return pd.DataFrame(results), sccs

# =====================================================
# 5. EXECUTION
# =====================================================

sim = CoffeeSimulator()
df, sccs = sim.run(DSM)

print("\n" + "="*75)
print("COFFEE SHOP OPERATIONS: KPI & REWORK ANALYSIS")
print("="*75)

print("\n1. KPI SCORECARD BY PROCESS STEP")
print(df[['Touchpoint', 'Feature', 'Value', 'Score', 'Metric']].to_string(index=False))

print("\n2. AGGREGATED BUSINESS HEALTH")
print(df.groupby('Metric')['Score'].mean().reset_index().round(3))

print("\n3. OPERATIONAL BOTTLENECKS (Coupled Production Knot)")
clusters = [list(c) for c in sccs if len(c) > 1]
if clusters:
    for i, c in enumerate(clusters):
        print(f"   ⚠️ Cluster {i+1} (Rework Loop): {c}")
else:
    print("   ✅ No complex rework loops detected.")

print("\n" + "="*75)

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

# Reset random seed for 2025
np.random.seed(2025)

# =====================================================
# 1. SERVICE TOUCHPOINTS (6 NODES)
# =====================================================
TOUCHPOINTS = [
    "Order_Intake",      # 0: Customer queuing & POS entry
    "Brewing_Prep",      # 1: Grinding & Espresso extraction
    "Milk_Texturing",    # 2: Frothing & Temp control
    "Quality_Audit",     # 3: Visual & Flavor verification
    "Handover_Vibe",     # 4: Customer name-call & interaction
    "Inventory_Log"      # 5: Waste tracking & stock update
]

# =====================================================
# 2. COFFEE DSM (DEPENDENCY STRUCTURE MATRIX)
# =====================================================
# High values = High likelihood of a step being impacted by another.
# Note the loop between 1, 2, and 3 (The "Production Knot")
DSM_MATRIX = np.array([
    #0    1    2    3    4    5
    [0,   1,   0,   0,   0,   0],   # 0: Order -> Prep
    [0,   0,   0.8, 0.9, 0,   0],   # 1: Prep -> Milk & Audit
    [0,   0.7, 0,   0.9, 0,   0],   # 2: Milk -> Prep (Feedback) & Audit
    [0,   0.5, 0.5, 0,   1,   0.4], # 3: Audit -> Rework Loop to 1/2 or Handover
    [0,   0,   0,   0,   0,   1],   # 4: Handover -> Inventory
    [0,   0,   0,   0,   0,   0],   # 5: Inventory (End)
])

# =====================================================
# 3. GRAPH ENGINE & MODULE DETECTION
# =====================================================
G = nx.DiGraph()
for i, u in enumerate(TOUCHPOINTS):
    for j, v in enumerate(TOUCHPOINTS):
        if DSM_MATRIX[i, j] > 0:
            G.add_edge(u, v, weight=DSM_MATRIX[i, j])

# Identify Rework Loops via Strongly Connected Components
SCCS = list(nx.strongly_connected_components(G))

# Topological Leveling for Modular Execution
module_map = {node: idx for idx, comp in enumerate(SCCS) for node in comp}
MG = nx.DiGraph()
for u, v in G.edges():
    mu, mv = module_map[u], module_map[v]
    if mu != mv: MG.add_edge(mu, mv)

LEVELS = list(nx.topological_generations(MG))

# =====================================================
# 4. DATA GENERATORS (Coffee Specifics)
# =====================================================
def gen_intake():
    return {"queue_length": np.random.poisson(5), "upsell_success": np.random.random() > 0.7}

def gen_prep():
    return {"extraction_time": np.random.normal(28, 3), "temp_celsius": np.random.uniform(90, 96)}

def gen_milk():
    return {"froth_density": np.random.beta(8, 2), "milk_waste_ml": np.random.exponential(20)}

def gen_audit():
    return {"defect_detected": np.random.choice([0, 1], p=[0.9, 0.1]), "aesthetic_score": np.random.uniform(0.7, 1.0)}

def gen_handover():
    return {"sentiment_score": np.random.uniform(3.5, 5.0), "wait_time_total": np.random.normal(180, 40)}

def gen_log():
    return {"inventory_accuracy": 0.99, "carbon_footprint_index": np.random.uniform(0.1, 0.5)}

GENERATOR_MAP = {
    "Order_Intake": gen_intake, "Brewing_Prep": gen_prep, "Milk_Texturing": gen_milk,
    "Quality_Audit": gen_audit, "Handover_Vibe": gen_handover, "Inventory_Log": gen_log
}

# =====================================================
# 5. SIMULATION & METRICS
# =====================================================
class CoffeeMetricsEvaluator:
    def compute_metrics(self, df):
        # Extract mean values for key indicators
        extract_time = df[df['variable'] == 'extraction_time']['value'].mean() or 28
        waste = df[df['variable'] == 'milk_waste_ml']['value'].mean() or 0
        sentiment = df[df['variable'] == 'sentiment_score']['value'].mean() or 4.0

        # Operational Risk: High waste and inconsistent extraction
        op_risk = (waste * 0.05) + (abs(28 - extract_time) * 0.1)
        # Quality Score: Sentiment and precision
        quality = (sentiment / 5.0) * 0.7 + (1.0 if 25 <= extract_time <= 31 else 0.5) * 0.3

        return np.clip(op_risk, 0, 1), np.clip(quality, 0, 1)

def run_modular_sim(levels, sccs):
    rows = []
    for lvl_idx, modules in enumerate(levels, 1):
        for m_idx in modules:
            nodes = list(sccs[m_idx])
            for tp in nodes:
                for k, v in GENERATOR_MAP[tp]().items():
                    rows.append({
                        "touchpoint": tp, "level": lvl_idx,
                        "type": "Knot (Rework)" if len(nodes) > 1 else "Linear",
                        "variable": k, "value": v
                    })
    return pd.DataFrame(rows)

# Execute
df_sim = run_modular_sim(LEVELS, SCCS)
evaluator = CoffeeMetricsEvaluator()
risk, qual = evaluator.compute_metrics(df_sim)

# =====================================================
# 6. REPORT
# =====================================================
print("\n=== COFFEE SHOP ARCHITECTURE OPTIMIZATION ===")
print(f"Total Workflow Steps: {len(TOUCHPOINTS)}")
print(f"System Coordination Complexity (DSM Sum): {DSM_MATRIX.sum():.2f}")

print("\n1. MODULAR STRUCTURE (Strongly Connected Components)")
for i, lvl in enumerate(LEVELS, 1):
    for mod_idx in lvl:
        nodes = list(SCCS[mod_idx])
        type_str = "⚠️ REWORK KNOT" if len(nodes) > 1 else "✅ LINEAR STEP"
        print(f"  Level {i} | {type_str:<15} | {nodes}")

print("\n2. SYSTEM HEALTH INDEX")
print(f"  Operational Risk Index: {risk:.4f} (Target: <0.2000)")
print(f"  Service Quality Score:  {qual:.4f} (Target: >0.8500)")

print("\n3. SAMPLE DATA (Knot Nodes)")
print(df_sim[df_sim['type'] == 'Knot (Rework)'].head(5).to_string(index=False))

In [ ]:
import numpy as np
import numpy as np

# ======================================================
# 1. METRIC KEYS (The 6-Node Coffee & Risk String)
# ======================================================
METRIC_KEYS = [
    "Order_Logistics",       # 0: Input Phase (Footfall & Queue)
    "Brewing_Hub",            # 1: HUB - Technical Extraction (The "MRI")
    "Safety_Audit_HACCP",    # 2: HUB - Regulatory Compliance
    "Liability_Underwriting",# 3: HUB - Financial Risk & Insurance
    "Service_Handover",      # 4: Experience & Sentiment
    "Aggregate_Margin"       # 5: Output Summary Index
]

# ======================================================
# 2. FEATURE KEYS (Operations + Actuarial Domain)
# ======================================================
FEATURE_KEYS = [
    # Node 0: Logistics
    'pedestrian_footfall', 'pos_latency_ms',
    # Node 1 & 2: Production/Safety Knot
    'boiler_pressure_stability', 'water_tds_level',
    'inspection_violation_count', 'sanitation_sop_score',
    # Node 3: Risk/Insurance
    'premium_per_revenue', 'liability_exposure_index',
    'risk_retention_limit', 'inventory_shrinkage_rate',
    # Node 4 & 5: Sentiment & Margin
    'sentiment_score', 'loyalty_opt_in',
    'cogs_reduction_target', 'waste_variance'
]

# ======================================================
# 3. MAPPING (Node -> Features)
# ======================================================
metric_feature_map = {
    "Order_Logistics":       ['pedestrian_footfall', 'pos_latency_ms'],
    "Brewing_Hub":           ['boiler_pressure_stability', 'water_tds_level'],
    "Safety_Audit_HACCP":    ['inspection_violation_count', 'sanitation_sop_score'],
    "Liability_Underwriting":['premium_per_revenue', 'liability_exposure_index'],
    "Service_Handover":      ['sentiment_score', 'loyalty_opt_in'],
    "Aggregate_Margin":      ['cogs_reduction_target', 'waste_variance']
}
# ======================================================
# 5. TARGET MATRIX (6x6 Interaction Graph)
# ======================================================
METRIC_TARGET = [
    # 0  1  2  3  4  5
    [1, 1, 0, 0, 0, 1], # 0: Logistics -> Prep & Margin
    [0, 1, 1, 0, 1, 1], # 1: Brewing -> Audit, Handover, Margin
    [0, 1, 1, 1, 0, 1], # 2: Audit -> Feedback to Prep, Risk, Margin
    [1, 0, 1, 1, 0, 1], # 3: Liability -> Feedbacks to Logistics & Audit
    [0, 0, 0, 0, 1, 1], # 4: Handover -> Margin
    [0, 0, 0, 0, 0, 1]  # 5: Aggregate
]

# Formulas (Actuarial & Operational Logic)
METRIC_FORMULAS = [
    lambda x: np.log(x + 1),            # 0. Logistics benefit
    lambda x: np.tanh(x) * 2.0,         # 1. Brewing Stability (The MEP of Coffee)
    lambda x: np.exp(-x * 3.0),         # 2. Safety (The Regulatory Accred of Coffee)
    lambda x: 1.0 / (x * 0.5 + 0.1),    # 3. Liability (Lower exposure = Higher Score)
    lambda x: x / 5.0,                  # 4. Sentiment (1-5 Scale)
    lambda x: np.mean(x)                # 5. Aggregate Roll-up
]
# ======================================================
# 6. OPTIMIZER (Branch & Bound)
# ======================================================
class BranchBoundOptimizer:
    def __init__(self, tol=1e-3, max_depth=20, minimize=True, value_range=(0.0, 10.0)):
        self.tol = tol
        self.max_depth = max_depth
        self.minimize = minimize
        self.value_range = value_range

    def optimize(self, features, y=None, metric_mask=None):
        y = np.zeros(3) if y is None else np.array(y[:3])
        base = np.mean(list(features.values())) + np.mean(y)

        a0, b0 = self.value_range
        a0 += base
        b0 += base

        work = [(a0, b0, 0)]
        best_x = None
        best_score = np.inf if self.minimize else -np.inf

        def better(s1, s2):
            return s1 < s2 if self.minimize else s1 > s2

        while work:
            a, b, depth = work.pop()
            mid = 0.5 * (a + b)

            # Apply Formula if mask is active
            mv = [f(mid) if m else 0.0 for f, m in zip(METRIC_FORMULAS, metric_mask)]
            score = sum(mv)

            if best_x is None or better(score, best_score):
                best_x = mid
                best_score = score

            if depth >= self.max_depth or (b - a) < self.tol:
                continue

            work.append((a, mid, depth + 1))
            work.append((mid, b, depth + 1))

        return best_x

# ======================================================
# 8. EVALUATOR
# ======================================================
class MetricsEvaluator:
    def __init__(
        self,
        data_matrix,
        metric_formulas=METRIC_FORMULAS,
        metric_feature_map=metric_feature_map,
        feature_keys=FEATURE_KEYS,
        feature_target=None,
        metric_target=None,
        tol=1e-3,
        max_depth=20,
        minimize=False,
        value_range=(0.0, 5.0)
    ):
        self.data_matrix = data_matrix
        self.metric_formulas = metric_formulas
        self.metric_feature_map = metric_feature_map
        self.feature_keys = feature_keys

        # Defaults
        self.feature_target = feature_target or [[1]*len(feature_keys) for _ in range(data_matrix.shape[0])]
        self.metric_target = metric_target or METRIC_TARGET
        self.num_nodes = data_matrix.shape[0]

        self.optimizer = BranchBoundOptimizer(
            tol=tol,
            max_depth=max_depth,
            minimize=minimize,
            value_range=value_range
        )

    def extract_features(self, node_idx):
        if node_idx >= len(self.data_matrix): return {}
        row = self.data_matrix[node_idx]
        mask = self.feature_target[node_idx]
        features = {k: v for k, v, m in zip(self.feature_keys, row, mask) if m}
        return features

    def compute_node_metrics(self, node_idx, y=None):
        features = self.extract_features(node_idx)
        metric_mask = self.metric_target[node_idx] if node_idx < len(self.metric_target) else [0]*len(METRIC_KEYS)
        metric_values = {}

        for key, formula, mask in zip(METRIC_KEYS, self.metric_formulas, metric_mask):
            if mask and key in self.metric_feature_map:
                relevant_keys = self.metric_feature_map[key]
                relevant_features = [features[f] for f in relevant_keys if f in features]
                x = np.mean(relevant_features) if relevant_features else 0.0

                # Optimize
                opt_value = self.optimizer.optimize(features={key: x}, y=y, metric_mask=[1])
                metric_values[key] = formula(opt_value)
            else:
                metric_values[key] = 0.0

        metric_values['score'] = sum(metric_values.values())
        return metric_values
# ======================================================
# 8. EXECUTION
# ======================================================

# Generate Mock Data (15 Nodes x 25 Features)
# ======================================================
# 9. DIMENSIONALITY CONFIGURATION (15 Nodes)
# ======================================================

# Dimensions assigned based on node complexity and coupling depth.
# "Hub" nodes and Financial nodes get higher dimensions.

# ======================================================
# 4. DIMENSIONALITY CONFIGURATION (6 Nodes)
# ======================================================
candidate_dims = [
    [4],   # 0: Location_Logistics (Linear Input)
    [16],  # 1: Brewing_Hub (HUB: High Variable Density)
    [16],  # 2: Safety_Audit_HACCP (HUB: Regulatory Risk)
    [16],  # 3: Liability_Underwriting (HUB: Financial Shield)
    [8],   # 4: Service_Handover (Coupled Experience)
    [4]    # 5: Aggregate_Margin (System Output)
]

D_graph = len(candidate_dims)

In [ ]:
import numpy as np
import pandas as pd

# Reset random seed for 2025
np.random.seed(2025)

# ======================================================
# 1. CORE SYSTEM CONFIGURATION (Names must be EXACT)
# ======================================================
# These names serve as the "Source of Truth" for the entire script.
TOUCHPOINTS = [
    "Order_Intake",      # 0
    "Brewing_Prep",      # 1
    "Milk_Texturing",    # 2
    "Quality_Audit",     # 3
    "Handover_Vibe",     # 4
    "Inventory_Log"      # 5
]

# Hub nodes (Brewing & Quality) require higher dimensions (16)
candidate_dims = [[4], [16], [8], [16], [8], [4]]

# ======================================================
# 2. SYNCHRONIZED GENERATOR MAP
# ======================================================
# FIX: Keys here must match TOUCHPOINTS exactly to avoid KeyError.
GENERATOR_MAP = {
    "Order_Intake": lambda: {
        "footfall": np.random.poisson(40),
        "pos_latency": np.random.exponential(5)
    },
    "Brewing_Prep": lambda: {
        "pressure": np.random.normal(9, 0.5),
        "temp": np.random.normal(92, 1.5),
        "extraction_vol": np.random.uniform(28, 32)
    },
    "Milk_Texturing": lambda: {
        "froth_quality": np.random.beta(8, 2),
        "steam_temp": np.random.normal(65, 2)
    },
    "Quality_Audit": lambda: {
        "violation_risk": np.random.beta(1, 10),
        "sanitation_score": np.random.uniform(80, 100)
    },
    "Handover_Vibe": lambda: {
        "sentiment": np.random.uniform(3, 5),
        "wait_time": np.random.rayleigh(120)
    },
    "Inventory_Log": lambda: {
        "cogs_ratio": np.random.uniform(0.2, 0.35),
        "labor_cost": np.random.normal(15, 2)
    }
}

# ======================================================
# 3. DATA MATRIX GENERATION & NORMALIZATION
# ======================================================
num_samples = 100
feature_list = []

for tp in TOUCHPOINTS:
    # Now correctly finds keys like 'Order_Intake'
    samples = [GENERATOR_MAP[tp]() for _ in range(num_samples)]
    df = pd.DataFrame(samples)

    # Handle categoricals if any
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.factorize(df[col])[0]
    feature_list.append(df)

DATA_MATRIX_RAW = pd.concat(feature_list, axis=1).to_numpy()
# Normalization logic
DATA_MATRIX = (DATA_MATRIX_RAW - DATA_MATRIX_RAW.min(axis=0)) / (np.ptp(DATA_MATRIX_RAW, axis=0) + 1e-8)

print(f"Success! Coffee Operational Matrix Created: {DATA_MATRIX.shape}")

# ======================================================
# 4. HIGH-FIDELITY SYNTHETIC TARGET GENERATOR
# ======================================================
def generate_coffee_targets(DATA_MATRIX, candidate_dims):
    dims_flat = [i[0] for i in candidate_dims]
    num_nodes = len(dims_flat)
    targets = []
    current_col = 0

    for node_idx in range(num_nodes):
        dim = dims_flat[node_idx]

        # Row selection: use the specific row index for the touchpoint
        row = DATA_MATRIX[node_idx % DATA_MATRIX.shape[0]]

        # Slicing the contiguous block for target signature
        block = row[current_col : current_col + dim]

        # HUB LOGIC: Reflective padding for 16D nodes
        if len(block) < dim:
            block = np.pad(block, (0, dim - len(block)), mode='reflect')

        targets.append({
            'node_id': node_idx,
            'touchpoint': TOUCHPOINTS[node_idx],
            'dim_required': dim,
            'target': np.round(block, 4)
        })

        # Modular wrap for feature variance recycling
        current_col = (current_col + dim) % DATA_MATRIX.shape[1]

    return targets

# Execute generation
synthetic_targets = generate_coffee_targets(DATA_MATRIX, candidate_dims)

# ======================================================
# 5. DIAGNOSTIC REPORT
# ======================================================
print(f"\n--- Coffee Operations Optimization Matrix (6-Node Compact) ---")
print(f"Total Operational Features Analyzed: {DATA_MATRIX.shape[1]}")
print("-" * 75)

for t in synthetic_targets:
    is_hub = "[HUB-KNOT]" if t['dim_required'] == 16 else "          "
    node_desc = f"{t['node_id']:01d} | {t['touchpoint']:<22} | {is_hub}"
    # Print the first 4 elements of the signature
    print(f"Node {node_desc} | Size: {t['dim_required']:02d} | Sig: {t['target'][:4]}...")

print("-" * 75)

__Replace above cells with Expert knowledge__
# Service Design Generator Simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
# Target Mask (which metrics apply to which node)

def top_k_masked_probs(weights, k):
    """
    Keep only top-k weights, zero out the rest, renormalize.
    """
    if k >= len(weights):
        return weights / (weights.sum() + 1e-12)

    idx = np.argpartition(weights, -k)[-k:]
    mask = np.zeros_like(weights)
    mask[idx] = weights[idx]

    s = mask.sum()
    if s > 0:
        mask /= s
    return mask
def get_all_paths(G, C, node_types, start=0, end=None):
    """
    Return all simple paths that respect coupling constraints.
    """
    if end is None:
        end = G.shape[0] - 1

    D = G.shape[0]
    paths = []

    def coupling_ok(i, j):
        label = C[i, j]

        # Default direct coupling
        if "->" in label and "->C" not in label:
            src, dst = label.split("->")
            return src == node_types[i] and dst == node_types[j]

        # Escalation case (B->B->C)
        if label == "B->B->C":
            return node_types[i] == "B" and node_types[j] == "B"

        return False

    def dfs(node, path, visited):
        if node == end:
            paths.append(path.copy())
            return

        for nxt in range(D):
            if G[node, nxt] > 0 and nxt not in visited:
                if not coupling_ok(node, nxt):
                    continue

                visited.add(nxt)
                dfs(nxt, path + [nxt], visited)
                visited.remove(nxt)

    dfs(start, [start], {start})
    return paths



# =============================================================================
# EXTERNAL DEPENDENCIES & CONFIGURATION
# =============================================================================
# These variables are referenced in the original code but not defined.
# Assumed to be present in the execution environment.
# -----------------------------------------------------------------------------
# D_graph = ...
# DATA_MATRIX = ...
# METRIC_KEYS = [...]
# METRIC_TARGET = [...]
# METRIC_FORMULAS = [...]
# METRIC_INVERSES = {...}
# synthetic_targets = ...
# MetricsEvaluator = ... (Class)
# -----------------------------------------------------------------------------
# =============================================================================
# NEW: DETERMINISTIC DFS & PATH EVALUATOR
# =============================================================================


def evaluate_fixed_paths(paths, node_metrics, beta=0.3):
    """
    Replaces collect_and_select_best_walks.
    Scores the specific paths found by DFS.
    """
    walks = []
    best = None

    for path in paths:
        cost = 0.0
        quality = 0.0

        # Calculate Path Metrics
        for node_idx in path:
            m = node_metrics[node_idx]
            cost += m.get("cost", 0.0)
            quality += m.get("quality", 0.0)

        score = quality - 1.0 * cost # using default lambda_cost=1.0

        # DFS paths are unique, so count is always 1
        count = 1
        adjusted_score = score / (1 + beta * count)

        record = {
            "path": path,
            "score": score,
            "adjusted_score": adjusted_score,
            "cost": cost,
            "quality": quality,
            "count": count
        }
        walks.append(record)

        if best is None or adjusted_score > best["adjusted_score"]:
            best = record

    return walks, best

def dsm_walk(G, node_metrics, start=0, max_steps=50, lambda_cost=1.0):
    D = G.shape[0]
    target_node = D - 1  # Define Nlast

    # Force start at 0 if not provided
    current = start
    path = [current]
    visited = {current}

    cost = 0.0
    quality = 0.0

    # Metrics for the start node
    m_start = node_metrics[current]
    cost += m_start.get("cost", 0.0)
    quality += m_start.get("quality", 0.0)

    reached_target = False

    for _ in range(max_steps):
        # If we reached the last node, stop successfully
        if current == target_node:
            reached_target = True
            break

        weights = np.zeros(D)

        for j in range(D):
            # Allow visiting the target even if visited (though unlikely to revisit in DAG)
            # But generally prevent cycles
            if j == 0:
                continue
            if j in visited:
                continue

            q = node_metrics[j].get("quality", 0.0)
            c = node_metrics[j].get("cost", 0.0)

            # Standard probability weight
            weights[j] = G[current, j] * (q / (1 + c))

        if weights.sum() == 0:
            break

        weights /= weights.sum()
        nxt = np.random.choice(D, p=weights)

        m = node_metrics[nxt]
        cost += m.get("cost", 0.0)
        quality += m.get("quality", 0.0)

        path.append(nxt)
        visited.add(nxt)
        current = nxt

    # Recalculate score based on success
    score = quality - lambda_cost * cost

    # Heavy penalty if the walk did not reach Nlast
    if not reached_target:
        score = -1e9

    return path, score, cost, quality

from collections import defaultdict

def collect_and_select_best_walks(G, node_metrics, n_walks=300, beta=0.3):
    walks = []
    best = None
    path_counts = defaultdict(int)
    D = G.shape[0]

    for _ in range(n_walks):
        # Force start at 0
        start = 0
        path, score, cost, quality = dsm_walk(
            G, node_metrics, start
        )

        # Only record if it successfully reached the last node
        # (Score is -1e9 if it failed, per updated dsm_walk)
        if path[-1] != (D - 1):
            continue

        key = tuple(path)
        path_counts[key] += 1

        # novelty penalty
        adjusted_score = score / (1 + beta * path_counts[key])

        record = {
            "path": path,
            "score": score,
            "adjusted_score": adjusted_score,
            "cost": cost,
            "quality": quality,
            "count": path_counts[key]
        }
        walks.append(record)

        if best is None or adjusted_score > best["adjusted_score"]:
            best = record

    return walks, best

class CouplingState:
    """
    One-step coupling automaton.
    C = transient escalation after B->B->C
    """

    def __init__(self):
        self.state = "NORMAL"

    def update(self, coupling):
        if coupling == "B->B->C":
            self.state = "C"
        else:
            self.state = "NORMAL"

    def allows(self, next_node_type):
        """
        Rule:
        C → random, but NOT B
        """
        if self.state == "C" and next_node_type == "B":
            return False
        return True

def infer_node_types(node_metrics):
    """
    A = quality-dominant
    B = cost-dominant
    """
    node_types = []
    for m in node_metrics:
        q = m.get("quality", 0.0)
        c = m.get("cost", 0.0)
        node_types.append("A" if q >= c else "B")
    return node_types

def build_coupling_matrix(node_types):
    """
    Builds a D x D coupling-label matrix.
    C exists only as a coupling escalation (B->B->C).
    """
    D = len(node_types)
    C = np.empty((D, D), dtype=object)

    for i in range(D):
        for j in range(D):
            src = node_types[i]
            dst = node_types[j]

            if src == "B" and dst == "B":
                C[i, j] = "B->B->C"
            else:
                C[i, j] = f"{src}->{dst}"

    return C


def coupling_weight(coupling_label, metrics_i, metrics_j):
    """
    OHMIC GF: I = ΔV / R
    Treats the transition as a vector field.
    """
    # Vector extraction
    v_i = np.array(list(metrics_i.values())) if isinstance(metrics_i, dict) else np.array(metrics_i)
    v_j = np.array(list(metrics_j.values())) if isinstance(metrics_j, dict) else np.array(metrics_j)

    # 1. Voltage (Potential Gain): Only conduct on improvement
    v_diff = np.maximum(v_j - v_i, 0)
    voltage = np.linalg.norm(v_diff) + 1e-6

    # 2. Resistance (Node Friction): Destination cost/complexity
    resistance = np.mean(v_j) + 0.1

    # 3. Current (Conductance)
    conductance = voltage / resistance

    # Type A = High Conductance, Type B = High Damping
    valve = 1.3 if "A" in coupling_label else 0.7
    return max(conductance * valve, 0.0001)

# Configuration
#candidate_dims = [[2], [2], [2], [2], [2], [2], [2], [2], [1]]
outer_generations = 1
inner_learning = 0.1
gamma_interlayer = 1
top_k = 21

# Initialize random state
np.random.seed()
seed = None  # Placeholder as per original logic

# Placeholder for Data Matrix generation (from original snippet)
# new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])


# =============================================================================
# HELPER CLASSES
# =============================================================================
class DSM_Tracker:
    """
    Tracks a DSM (Design Structure Matrix) layer and its residual.
    """
    def __init__(self, multiplex_layer):
        self.layer = multiplex_layer
        self.primary_dsm = None
        self.residual_dsm = None
        self.update_dsms()

    def update_dsms(self):
        if self.primary_dsm is None:
            # First time: store current DSM as reference
            self.primary_dsm = self.layer.chosen_Gmat.copy()
            self.residual_dsm = np.zeros_like(self.primary_dsm)
        else:
            # Update residual: current DSM minus primary
            current = self.layer.chosen_Gmat
            self.residual_dsm = current - self.primary_dsm

    def get_matrices(self):
        return self.primary_dsm, self.residual_dsm

    def print_matrices(self):
        print("\n--- Primary DSM ---")
        print(self.primary_dsm)
        print("\n--- Residual DSM ---")
        print(self.residual_dsm)


class DSM_Layer_Decomposer:
    """
    Manages the additive decomposition of DSM layers.
    """
    def __init__(self, baseline_matrix, mode='additive'):
        self.baseline_matrix = baseline_matrix.copy()
        self.current_total = baseline_matrix.copy()
        self.mode = mode
        self.layers = []
        self.residuals = []

    def add_snapshot(self, new_total_matrix):
        """
        Calculates the DELTA (change) between the new state and the previous state,
        stores that delta as a layer.
        """
        delta = new_total_matrix - self.current_total
        self.layers.append(delta.copy())

        # Update current tracker
        self.current_total = new_total_matrix.copy()

        # Calculate residual (Difference from the baseline)
        residual = self.current_total - self.baseline_matrix
        self.residuals.append(residual)

        layer_id = len(self.layers) - 1
        print(f"\n=== DSM LAYER {layer_id} CAPTURED ===")
        print(f"Layer Contribution (Delta):\n{np.round(delta, 3)}")

        return delta

    def get_reconstruction(self):
        return np.sum(self.layers, axis=0)


class SVM:
    """
    Metric-inverse multi-output SVM with epsilon-insensitive loss.
    """
    def __init__(self, input_dim, output_dim=None, metric_keys=None, lr=0.001, epsilon=0.1):
        self.input_dim = input_dim
        self.metric_keys = metric_keys

        if metric_keys is not None:
            self.output_dim = len(metric_keys)
        elif output_dim is not None:
            self.output_dim = output_dim
        else:
            self.output_dim = 1

        self.lr = lr
        self.epsilon = epsilon

        # Lazy initialization
        self.X_train = None
        self.y_train = None
        self.alpha = None
        self.b = None

    def train_step(self, X, y_true):
        X = np.array(X)
        y_true = np.array(y_true)
        n_samples = X.shape[0]

        if self.X_train is None:
            self.X_train = X.copy()
            self.y_train = y_true.copy()
            self.alpha = np.zeros((X.shape[1], self.output_dim))
            self.b = np.zeros(self.output_dim)

        # Linear kernel
        y_pred = X.dot(self.alpha) + self.b

        # Epsilon-insensitive loss
        diff = y_pred - y_true
        mask = np.abs(diff) > self.epsilon
        diff *= mask

        grad_alpha = X.T.dot(diff) / n_samples
        grad_b = diff.mean(axis=0)

        self.alpha -= self.lr * grad_alpha
        self.b -= self.lr * grad_b

        loss = np.mean(np.maximum(0, np.abs(y_pred - y_true) - self.epsilon))
        return loss

    def predict(self, X):
        X = np.array(X)
        y_pred = X.dot(self.alpha) + self.b

        if self.metric_keys is None:
            return y_pred

        # Apply metric inverses
        y_transformed = np.zeros_like(y_pred)
        for i, key in enumerate(self.metric_keys):
            inverse_fn = METRIC_INVERSES[key]
            # Handle potential list return from inverse_fn
            val_func = lambda y: inverse_fn(y)[0] if isinstance(inverse_fn(y), list) else inverse_fn(y)
            y_transformed[:, i] = np.array([val_func(y) for y in y_pred[:, i]])

        return y_transformed


class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        if seed is not None:
            np.random.seed(seed)

        self.D_graph = D_graph
        self.edge_threshold = edge_threshold
        self.gamma = gamma

        # Handle list vs int inputs for dimensions
        m_dim = max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim

        if inter_dim is not None:
            self.inter_dim = inter_dim[0] if isinstance(inter_dim, list) else inter_dim
        else:
            self.inter_dim = m_dim

        self.max_input = 2 * m_dim

        # Initialize weights
        self.weights = {}
        self.bias = {}
        for i in range(D_graph):
            for j in range(D_graph):
                if i != j:
                    w_init = np.random.uniform(-0.1, 0.1, (self.inter_dim, self.max_input))
                    self.weights[(i, j)] = w_init
                    self.bias[(i, j)] = np.zeros(self.inter_dim)

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        # Pad or truncation
        if len(concat) < self.max_input:
            concat = np.pad(concat, (0, self.max_input - len(concat)))
        else:
            concat = concat[:self.max_input]

        # Normalize
        concat = (concat - np.mean(concat)) / (np.std(concat) + 1e-12)

        # Activation
        v = self.weights[(i, j)].dot(concat) + self.bias[(i, j)]
        input_strength = np.clip(np.mean(np.abs(concat)), 0, 1)
        v = v * input_strength

        return 1 / (1 + np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i == j:
                    continue
                if abs(Gmat[i, j]) > self.edge_threshold:
                    acts[(i, j)] = self.compute_edge_activation(i, j, nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts) < 2:
            return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1, keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2, axis=1) / (A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds, stds) + 1e-12)
        np.fill_diagonal(corr, 0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat, nested_reps)
        if not acts:
            return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

def build_dsm_from_walks(D, paths):
    """
    Constructs a DSM where entry [i,j] is the probability
    that a successful process moves from i to j.
    """
    flow = np.zeros((D, D))

    # Count transitions
    for path in paths:
        for k in range(len(path) - 1):
            u, v = path[k], path[k+1]
            flow[u, v] += 1

    # Normalize by the total number of successful walks.
    # This prevents 'saturation'—if an edge is rarely used, it stays small.
    n_paths = len(paths)
    if n_paths > 0:
        flow = flow / n_paths

    np.fill_diagonal(flow, 0.0)
    return flow
# =============================================================================
# MAIN OPTIMIZER CLASS
# =============================================================================
class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, max_steps=50):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps

    def transition_probs(self, i, k=2):
        w = np.zeros(self.D)

        for j in range(self.D):
            if i == j:
                continue

            weight = coupling_weight(self.C[i, j], self.node_metrics[j])

            # Directional heuristic (keep yours)
            if j > i:
                weight *= 1.2
            elif j < i:
                weight *= 0.8

            w[j] = weight

        # If nothing viable, fallback to uniform
        if w.sum() == 0:
            return np.ones(self.D) / self.D

        # Emphasize strong edges
        w = w ** 2

        # 🔒 HARD SPARSITY CONSTRAINT (TOP-K)
        w = top_k_masked_probs(w, k)

        return w


    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        for _ in range(self.max_steps):
            if current == target_node:
                break

            probs = self.transition_probs(current)

            # Move
            current = np.random.choice(self.D, p=probs)
            path.append(current)

            if current == target_node:
                break

        return path
class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, max_steps=50, top_k=2):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps
        self.top_k = top_k  # <--- FIX: Store the parameter

    def transition_probs(self, i):
        w = np.zeros(self.D)

        for j in range(self.D):
            if i == j:
                continue

            weight = coupling_weight(self.C[i, j], self.node_metrics[j])

            # Directional heuristic
            if j > i:
                weight *= 1.2
            elif j < i:
                weight *= 0.8

            w[j] = weight

        if w.sum() == 0:
            return np.ones(self.D) / self.D

        w = w ** 2

        # 🔒 FIX: Use the stored self.top_k
        w = top_k_masked_probs(w, self.top_k)

        return w

    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        for _ in range(self.max_steps):
            if current == target_node:
                break

            probs = self.transition_probs(current)

            # Move
            current = np.random.choice(self.D, p=probs)
            path.append(current)

            if current == target_node:
                break

        return path
class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, metric_targets, max_steps=100, top_k=2):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.metric_targets = metric_targets
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps
        self.top_k = top_k

        # ⚡ Precompute the base Ohmic Conductance (The 'Clean' Pipe)
        self.Gamma_base = self._precompute_conductance()

    def _precompute_conductance(self):
        gamma_mat = np.zeros((self.D, self.D))
        for i in range(self.D):
            for j in range(self.D):
                if i != j and self.metric_targets[i][j] != 0:
                    # Calculate I = V / R using the means of the metric vectors
                    gamma_mat[i, j] = coupling_weight(self.C[i, j], self.node_metrics[i], self.node_metrics[j])
        return gamma_mat

    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        for _ in range(self.max_steps):
            if current == target_node:
                break

            # 1. Grab precomputed conductances for 'current'
            w = self.Gamma_base[current, :].copy()

            # 2. APPLY MEANS-BASED FATIGUE (The Singularity Breaker)
            # We penalize nodes based on the MEAN of their appearance in the path
            for j in range(self.D):
                if j in path:
                    # Every visit reduces the conductance by a mean-factor
                    # This increases the 'Resistance' of the loop
                    count = path.count(j)
                    w[j] *= (1.0 / (1.0 + count))

            # 3. DIRECTIONAL BIAS (The Gravity Jump)
            # Ensure the jump 'means' something by favoring forward progress
            indices = np.arange(self.D)
            w[indices > current] *= 1.5  # Potential to jump ahead
            w[indices < current] *= 0.2  # Heavy friction for jumping back

            # 4. EXECUTE THE JUMP
            if w.sum() == 0:
                break # Circuit broken

            # Sharpen the distribution and pick the best K
            probs = top_k_masked_probs(w**2, self.top_k)
            current = np.random.choice(self.D, p=probs)
            path.append(current)

        return path

import numpy as np

class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, metric_targets, max_steps=100, top_k=2):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.metric_targets = metric_targets
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps
        self.top_k = top_k

        # 1. Internal Enigma State (The 'Rotor' Configuration)
        # This permutation vector re-maps the node indices on every jump
        self.enigma_state = np.arange(self.D)

        # 2. Precompute the Base Ohmic Conductance (Static Wiring)
        self.Gamma_base = self._precompute_conductance()

    def _precompute_conductance(self):
        """Standard Ohmic Flow: Current = Voltage / Mean(Metrics)"""
        gamma_mat = np.zeros((self.D, self.D))
        for i in range(self.D):
            for j in range(self.D):
                if i != j and self.metric_targets[i][j] != 0:
                    # Ohmic GF: delta Quality / Mean Friction
                    gamma_mat[i, j] = coupling_weight(self.C[i, j], self.node_metrics[i], self.node_metrics[j])
        return gamma_mat

    def rotate_rotors(self, current_node):
        """
        The Enigma Shift: Permutes the state based on the 'Mean' of the node.
        This changes the available 'exits' for the next step.
        """
        metrics = self.node_metrics[current_node]
        # Use the mean as the 'Notch' on the rotor
        mean_val = np.mean(list(metrics.values())) if isinstance(metrics, dict) else np.mean(metrics)

        # Shift the state machine by the intensity of the current node
        shift = int(mean_val * 10) % self.D
        self.enigma_state = np.roll(self.enigma_state, shift)

    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        for step in range(self.max_steps):
            if current == target_node:
                break

            # ⚡ STEP 1: Rotate the Enigma State Machine
            self.rotate_rotors(current)

            # ⚡ STEP 2: Get Base Weights
            w = self.Gamma_base[current, :].copy()

            # ⚡ STEP 3: Permutation Mapping
            # Scramble the available flows through the current Enigma configuration
            # This makes the 'Singularity' impossible because the weights shift every visit
            w = w[self.enigma_state]

            # ⚡ STEP 4: Directional Bias (Inertia)
            indices = np.arange(self.D)
            w[indices > current] *= 2.0  # Forward Pull
            w[indices < current] *= 0.1  # Backward Resistance (Rework)

            if w.sum() == 0:
                break # Open Circuit

            # ⚡ STEP 5: Jump
            probs = top_k_masked_probs(w**2, self.top_k)
            current = np.random.choice(self.D, p=probs)
            path.append(current)

        return path

class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, metric_targets, max_steps=100, top_k=2):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.metric_targets = metric_targets
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps
        self.top_k = top_k

        # ⚡ PRE-COMPUTED MANIFOLD (The Speed Fix)
        # We build the 'Enigma' logic directly into the static weights
        self.Gamma_Enigma = self._precompute_enigma_manifold()

    def _precompute_enigma_manifold(self):
        """
        Calculates the entire state-space once.
        Incorporates 'Mean Resistance' into the structure.
        """
        manifold = np.zeros((self.D, self.D))
        for i in range(self.D):
            m_i = self.node_metrics[i]
            # Mean Friction of source node acts as a Rotor Notch
            friction_i = np.mean(list(m_i.values())) if isinstance(m_i, dict) else np.mean(m_i)

            for j in range(self.D):
                if i != j and self.metric_targets[i][j] != 0:
                    # Ohmic Conductance
                    base_i = coupling_weight(self.C[i, j], m_i, self.node_metrics[j])

                    # ENIGMA BIAS: Favor jumps that 'match' the rotor phase
                    # This replaces the slow np.roll() inside the run loop
                    phase_shift = (i + j) % self.D
                    enigma_mod = 1.0 + (0.1 * np.sin(phase_shift * friction_i))

                    # GRAVITY BIAS: Forward progress is structurally cheaper
                    gravity = 2.0 if j > i else 0.2

                    manifold[i, j] = base_i * enigma_mod * gravity
        return manifold

    def run(self, start=0):
        path = [start]
        current = start
        target = self.D - 1

        # Pre-slice Gamma to avoid lookups
        G = self.Gamma_Enigma

        for _ in range(self.max_steps):
            if current == target:
                break

            # ⚡ O(1) LOOKUP: No math, no counts, no state shifts
            w = G[current].copy()

            # Apply a light novelty penalty only if we hit a loop
            if current in path[:-1]:
                w *= 0.1 # Instant circuit breaker

            if w.sum() == 0:
                break

            # Probabilistic Jump
            probs = top_k_masked_probs(w**2, self.top_k)
            current = np.random.choice(self.D, p=probs)
            path.append(current)

        return path

In [ ]:
try:
    D_GRAPH = D_graph
except:
    D_graph=D_graph
def build_coupling_weight(label, m_i, m_j):
    """
    Interpretation: Pascal's Law P = F / A
    Force (F) = Potential delta in metrics (Improvement)
    Area (A) = Complexity/Friction of the destination node
    Valve = Orifice efficiency based on coupling type
    """
    # 1. THE NODAL VECTORS
    v_i = np.array(list(m_i.values())) if isinstance(m_i, dict) else np.array(m_i)
    v_j = np.array(list(m_j.values())) if isinstance(m_j, dict) else np.array(m_j)

    # 2. THE DRIVING FORCE (F)
    # The pressure only builds if the destination offers more "potential"
    # F = || max(0, v_j - v_i) ||
    force = np.linalg.norm(np.maximum(v_j - v_i, 0)) + 1e-9

    # 3. THE SURFACE AREA (A)
    # The 'wider' the destination node (more complex/costly),
    # the more the force is distributed, lowering the pressure.
    surface_area = np.mean(v_j) + 0.1

    # 4. THE VALVE ORIFICE (Efficiency)
    # A-type couplings are 'Wide Nozzles' (High pressure translation)
    # B-type couplings are 'Constricted Nozzles' (Damped flow)
    label_str = str(label) if label is not None else ""
    if "A" in label_str:
        motivator = force # High flow efficiency
    elif "B" in label_str:
        motivator =  1.0  # Damped efficiency
    else:
        motivator = surface_area# Standard atmospheric pressure

    # 5. THE SYSTEM PRESSURE (P)
    # (i) Power driver
    # (iii) Force driver
    # (ii) Pressure
    pressure = motivator * (force / surface_area)

    return max(pressure, 0.0001)
def infer_node_types(node_metrics):
    """
    Determines node type based on Cost vs Quality dominance.
    A = quality-dominant, B = cost-dominant.
    """
    node_types = []
    for m in node_metrics:
        # Handle dict or list input
        if isinstance(m, dict):
            q = m.get("quality", 0.0)
            c = m.get("cost", 0.0)
        else:
            # Assuming [cost, quality] or similar if list,
            # but defaulting to A if structure unknown
            q, c = 1, 0
        node_types.append("A" if q >= c else "B")
    return node_types

def build_coupling_matrix(node_types):
    """
    Builds a D x D coupling-label matrix.
    C exists only as a coupling escalation (B->B->C).
    """
    D = len(node_types)
    C = np.empty((D, D), dtype=object)

    for i in range(D):
        for j in range(D):
            src = node_types[i]
            dst = node_types[j]

            if src == "B" and dst == "B":
                C[i, j] = "B->B->C"
            else:
                C[i, j] = f"{src}->{dst}"
    return C

def build_dsm_from_walks(D, paths):
    """
    Constructs a DSM (Design Structure Matrix) from walk paths.
    Entry [i,j] is the probability that a process moves from i to j.
    """
    flow = np.zeros((D, D))
    for path in paths:
        for k in range(len(path) - 1):
            u, v = path[k], path[k+1]
            flow[u, v] += 1

    n_paths = len(paths)
    if n_paths > 0:
        flow = flow / n_paths

    np.fill_diagonal(flow, 0.0)
    return flow

# =============================================================================
# 2. THE METRIC-DRIVEN RANDOM WALKER (Consolidated)
# =============================================================================

class CouplingState:
    """
    One-step coupling automaton to enforce B->B->C rules.
    """
    def __init__(self):
        self.state = "NORMAL"

    def update(self, coupling):
        if coupling == "B->B->C":
            self.state = "C"
        else:
            self.state = "NORMAL"

    def allows(self, next_node_type):
        # Rule: After C state, random is allowed, but strictly NOT B
        if self.state == "C" and next_node_type == "B":
            return False
        return True

class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, metric_targets=None, max_steps=50, top_k=2, temperature=1.0):
        """
        Unified Walker Class.
        :param coupling_matrix: DxD matrix of transition labels.
        :param node_metrics: List of dicts/vectors for node values.
        :param metric_targets: (Optional) Binary mask or Target Matrix.
        :param max_steps: Maximum path length.
        :param top_k: Branching factor constraint.
        :param temperature: Control randomness (Higher = more random, Lower = more deterministic).
        """
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.node_types = infer_node_types(node_metrics)

        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps
        self.top_k = top_k
        self.temperature = temperature

        # Handle Mask: If None, allow all connections (ones)
        if metric_targets is not None:
            self.mask = np.array(metric_targets)
        else:
            self.mask = np.ones((self.D, self.D))

    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        # Initialize Automaton for this specific run
        automaton = CouplingState()

        for _ in range(self.max_steps):
            if current == target_node:
                break

            # 1. Calculate Raw Weights (Ohmic Conductance)
            weights = np.zeros(self.D)
            for j in range(self.D):
                if current == j: continue

                # A. Structural Mask Check (User provided mask)
                if self.mask[current, j] == 0:
                    continue

                # B. Automaton Rule Check
                if not automaton.allows(self.node_types[j]):
                    continue

                # C. Compute Physics-based Weight
                base_w = build_coupling_weight(self.C[current, j], self.node_metrics[current], self.node_metrics[j])

                # D. Directional Bias (Gravity)
                gravity = 1.5 if j > current else 0.4

                # E. Loop Friction (Novelty check)
                friction = 0.1 if j in path else 1.0

                weights[j] = base_w * gravity * friction

            # 2. Check for Dead End
            if weights.sum() == 0:
                break

            # 3. Apply Temperature (Softmax-ish scaling) for True Randomness
            # Normalize first to avoid overflow in exp
            weights = weights / (weights.max() + 1e-12)
            exp_weights = np.exp(weights / self.temperature)

            # Zero out the ones that were originally zero
            exp_weights[weights == 0] = 0

            # 4. Top-K Sparsity & Normalization
            probs = top_k_masked_probs(exp_weights, self.top_k)

            # 5. Stochastic Jump
            nxt = np.random.choice(self.D, p=probs)

            # Update state
            automaton.update(self.C[current, nxt])
            path.append(nxt)
            current = nxt

        return path
from collections import deque

        #return True
# --- Inner Loop (FCM & Learning) ---
INNER_FCM_STEPS = 1000       # Iterations per node simulation
INNER_LR_X = 1.0             # Learning rate for State X
INNER_LR_Y = 0.01           # Learning rate for State Y
INNER_LR_W = 1.0             # Learning rate for Weights
INNER_SVM_LR = 0.01          # SVM Learning Rate
INNER_GAMMA = 1.0            # Inter-layer neural connection strength

# --- Random Walk & Pathfinding ---
WALK_BETA = 0.3              # Novelty penalty (dampens repeated paths)
WALK_LAMBDA_COST = 1.0       # Penalty weight for cost metrics

# --- Outer Loop (Topology Optimization) ---
OUTER_GENERATIONS = 1        # Iterations per Layer
OUTER_COST_LIMIT = 1000      # Normalization ceiling for scores
INTER_EDGE_THRESH = 0.02     # Min DSM weight to trigger neural link

for ijk in range(D_GRAPH):
    for jik in range(5,30):
        print(50*'_',ijk,50*'-',jik,50*'=')
        #===============================================================================
        OUTER_N_SIMS = 1000          # More simulations to find the "Hidden Gem" paths
        WALK_MAX_STEPS = 100          # Let the walker explore complex relationships deeply
        DSM_TARGET_EDGES = jik        # Allow HIGHER density (Complexity is allowed!)
        OUTER_DSM_LAYERS = 1         # Balanced hierarchy (Structure -> Systems -> Skin)
        DSM_ADDITIVE_RATE = 0.45      # Low Learning Rate: Learn slowly, don't panic.
        DSM_FEEDBACK_STR = 0.05      # Weak Feedback: Listen to problems, but don't obsess.
        WALK_TOP_K = 2               # Soft Sparsity: Consider more options per step.
        DSM_FEEDBACK_FILTER = 0.1    # Only react to major issues.
        DSM_PRUNE_THRESH = 0.02      # Keep subtle connections.
        DSM_INIT_RANGE = 0.2         # Start with a blanker slate.
        STARTING_POINT = ijk           # START AT SITE ANALYSIS (Respect the Land).
        #=============================================================================
        class Fuzzy_Hierarchical_Multiplex:
            def __init__(self, candidate_dims, D_graph,
                        synthetic_targets,
                        gamma_interlayer=1.0, causal_flag=False,
                        metrics=METRIC_KEYS, metric_mask=METRIC_TARGET):

                self.candidate_dims = candidate_dims
                self.D_graph = D_graph
                self.synthetic_targets = synthetic_targets
                self.causal_flag = causal_flag
                self.best_dim_per_node = [len(t)-1 for t in synthetic_targets]
                self.MM = metric_mask
                self.MK = metrics
                self.MKI = metrics + ['score']

                self.PLM = [[] for _ in range(self.D_graph)]
                self.PLMS = [[] for _ in range(self.D_graph)]
                self.nested_reps = [np.zeros(c[0]) for c in candidate_dims]

                # Inter-layer setup
                self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
                self.chosen_Gmat = np.random.uniform(0.0, 0.3, (D_graph, D_graph))
                np.fill_diagonal(self.chosen_Gmat, 0)

                self.l2_before, self.l2_after = [], []
                self.max_target_len = max(len(t['target']) for t in synthetic_targets)
                self.svm_lr = 0.01

                self.metric_traces = {k: [] for k in metrics}
                self.metric_traces_per_node = [{} for _ in range(self.D_graph)]

                # DSM optimization hyperparameters
                self.dsm_lr = 0.1
                self.dsm_l1 = 0.02
                self.dsm_clip = 1.0
                self.dsm_history = []
                self.dsm_cost_weight = 0.05

            def print_dsm_basic(self):
                D = self.D_graph
                print("\n=== DESIGN STRUCTURE MATRIX (DSM) : Gmat ===")
                header = "     " + " ".join([f"N{j:>4}" for j in range(D)])
                print(header)
                for i in range(D):
                    row = "N{:>2} | ".format(i)
                    for j in range(D):
                        row += f"{self.chosen_Gmat[i, j]:>5.2f} "
                    print(row)

            # ---------- INNER LOOP (FCM) ----------
            def run_inner(self, node_idx, target, D_fcm,
                        steps=INNER_FCM_STEPS, lr_x=INNER_LR_X, lr_y=INNER_LR_Y, lr_W=INNER_LR_W,
                        decorrelate_metrics=False):

                # --- Initialize activations ---
                x = np.random.uniform(-0.6, 0.6, D_fcm)
                y = np.random.uniform(-0.1, 0.1, D_fcm)

                # L2 tracking
                self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx][:len(target)] - target))

                # --- FCM updates ---
                W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
                np.fill_diagonal(W, 0)

                for _ in range(steps):
                    z = y.dot(W) + x
                    Theta_grad_z = z - target
                    Theta_grad_x = Theta_grad_z
                    Theta_grad_y = Theta_grad_z.dot(W.T)
                    Theta_grad_W = np.outer(y, Theta_grad_z)

                    x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
                    y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)
                    W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)

                    x = np.clip(x, 0, 1)
                    y = np.clip(y, 0, 1)
                    np.fill_diagonal(W, 0)
                    W = np.clip(W, -1, 1)

                # --- Update nested representation ---
                self.nested_reps[node_idx][:len(x)] = x
                self.l2_after.append(np.linalg.norm(x - target))

                # --- Extract node features ---
                # Assuming MetricsEvaluator is a global or imported class
                metrics_evaluator = MetricsEvaluator(data_matrix=DATA_MATRIX)
                features = metrics_evaluator.extract_features(node_idx)
                feat_vals = np.array(list(features.values()))

                # --- Compute metrics scaled by activations + features ---
                metric_mask = METRIC_TARGET[node_idx]
                metric_values = {}

                for key, formula, mask in zip(METRIC_KEYS, METRIC_FORMULAS, metric_mask):
                    if mask:
                        weighted_input = np.mean(feat_vals)
                        # Outer scale check
                        outer_scale = getattr(self, 'best_node_weights', {}).get(node_idx, 1.0)
                        if isinstance(outer_scale, (list, np.ndarray)):
                            # fallback if it was stored incorrectly in previous context
                            outer_scale = 1.0

                        weighted_input *= outer_scale
                        metric_val = formula(weighted_input)
                        metric_values[key] = metric_val

                        # STORE DATAPOINT
                        self.metric_traces[key].append((weighted_input, metric_val))
                    else:
                        metric_values[key] = 0.0

                # --- Total score ---
                metric_values['score'] = sum(metric_values.values())

                # --- Build SVM Training Data ---
                metric_output_vals = np.array(
                    [v for k, v in metric_values.items() if k not in ['score', 'x', 'feat_vals']]
                )

                # Lazy init per-node SVM
                if not hasattr(self, "node_svms"):
                    self.node_svms = {}

                if node_idx not in self.node_svms:
                    self.node_svms[node_idx] = SVM(
                        input_dim=len(self.MK),
                        output_dim=self.candidate_dims[node_idx][0],
                        lr=self.svm_lr
                    )

                svm = self.node_svms[node_idx]

                # Build SVM Input/Output
                x_in_full = np.zeros(len(self.MK))
                x_in_full[:len(metric_output_vals)] = metric_output_vals
                x_in = x_in_full.reshape(1, -1)

                y_out_full = np.zeros(self.candidate_dims[node_idx][0])
                y_out_full[:len(x)] = x
                y_out = y_out_full.reshape(1, -1)

                # Train SVM
                _ = svm.train_step(x_in, y_out)

                # --- Store PLMS trace ---
                self.PLMS[node_idx].append((float(weighted_input), metric_output_vals))

                if len(self.PLMS[node_idx]) % 100 == 0:
                    print(f"Node {node_idx}, samples learned:", len(self.PLMS[node_idx]))

                # --- Compute inter-layer MI ---
                mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

                return x, y, W, mi_score, metric_values

            # ---------- OUTER LOOP (Topology Optimization) ----------
            def run_outer(self, outer_cost_limit=OUTER_COST_LIMIT, alpha=0.0, additive_rate=DSM_ADDITIVE_RATE, n_simulations=OUTER_N_SIMS):
                """
                CORRECTED LOGIC (Distributed Flow):
                1. Uses Random Walk to find high-probability metric flows starting from ANY node.
                2. Injects FEEDBACK LOOPS to create Coupled Blocks.
                3. Prunes weak edges to prevent 'Total Chaos'.

                *Update:* Removed 'Start at 0' handicap. Now samples flows from all subsystems.
                """
                node_metrics_list = self.capped_node_metrics
                D = self.D_graph

                # =========================================================
                # 1. METRIC SCORING
                # =========================================================
                raw_scores = np.array([m['score'] for m in node_metrics_list])
                total_raw = raw_scores.sum()
                if total_raw > outer_cost_limit:
                    scale_factor = outer_cost_limit / total_raw
                    for metrics in node_metrics_list:
                        for key in self.MKI:
                            metrics[key] *= scale_factor
                    raw_scores *= scale_factor

                fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=False)
                self.weighted_fmt = fuzzy_tensor.copy()

                # Calculate Contributions
                node_contributions = np.zeros(D)
                for i in range(D):
                    own_score = raw_scores[i]
                    fmt_contrib = fuzzy_tensor[i, :, :].sum() - fuzzy_tensor[i, i, :].sum()
                    node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib
                self.node_score_contributions = node_contributions
                self.correlation_penalty = 0.0

                # =========================================================
                # 2. PROBABILISTIC PATHFINDING (Distributed Random Walk)
                # =========================================================

                node_metrics = self.capped_node_metrics
                node_types = infer_node_types(node_metrics)
                C_matrix = build_coupling_matrix(node_types)
                print(node_types)
                # Initialize Walker
                # Inside run_outer method:

# ... (Previous metric scoring logic) ...

                # Pass the mask (METRIC_TARGET) explicitly
                # Assuming 'self.MM' holds the METRIC_TARGET data structure you provided
                walker = MetricDrivenRandomWalk(
                    C_matrix,
                    node_metrics,
                    metric_targets=self.MM, # <--- PASS THE MASK HERE
                    max_steps=WALK_MAX_STEPS,
                    top_k=WALK_TOP_K
                )

# ... (Rest of pathfinding logic) ...

                # --- VALIDATOR (RELAXED) ---
                                # 1. Define the Tiers of Validation
                def is_path_valid(path):
                    # 1. Topology Checks
                    if len(path) < 2: return False        # A path must go somewhere
                    if path[-1] != (D - 1): return False  # Must still converge to Project Completion
                    if len(set(path)) != len(path): return False # No self-cycles

                    # 2. Coupling Constraints
                    state_machine = CouplingState()
                    for k in range(len(path) - 1):
                        u, v = path[k], path[k+1]
                        type_u, type_v = node_types[u], node_types[v]

                        if not state_machine.allows(type_v): return False

                        coupling = "B->B->C" if (type_u == "B" and type_v == "B") else f"{type_u}->{type_v}"
                        state_machine.update(coupling)
                    return True


                # --- DISTRIBUTED SAMPLING ---
                valid_paths = []
                print(f" [Optimizer] Sampling {n_simulations} distributed paths (Any Start -> End)...")

                # Potential start nodes: 0 to D-2 (Any node except the final Sink node)
                possible_starts = list(range(D - 1))

                for _ in range(n_simulations):
                    # Randomly select a starting subsystem to ensure "All Processes Included"

                    path = walker.run(start=STARTING_POINT)

                    if is_path_valid(path):
                        valid_paths.append(path)

                # FALLBACK
                if len(valid_paths) == 0:
                    print(" [WARNING] Strict constraints failed. Synthesizing default backbone.")
                    valid_paths.append(list(range(D)))

                # Deduplicate
                unique_paths = sorted(list(set(tuple(p) for p in valid_paths)), key=lambda x: len(x), reverse=True)
                valid_paths = [list(p) for p in unique_paths]

                # Deduplicate
                unique_paths = sorted(list(set(tuple(p) for p in valid_paths)), key=lambda x: len(x), reverse=True)
                valid_paths = [list(p) for p in unique_paths]

                # =========================================================
                # 3. LAYER CONSTRUCTION & FEEDBACK INJECTION
                # =========================================================

                G_layer = build_dsm_from_walks(D, valid_paths)

                # Feedback Loops
                feedback_strength = DSM_FEEDBACK_STR
                G_feedback = G_layer.T * feedback_strength

                # Filter feedback
                G_feedback[G_layer < DSM_FEEDBACK_FILTER] = 0.0

                # Combine
                G_layer_final = G_layer + G_feedback

                # =========================================================
                # 4. UPDATE, AMPLIFY & PRUNE
                # =========================================================

                self.chosen_Gmat = self.chosen_Gmat + (additive_rate * G_layer_final)

                if np.max(self.chosen_Gmat) > 0:
                    self.chosen_Gmat /= np.max(self.chosen_Gmat)

                # Top-K Pruning
                TARGET_EDGES = DSM_TARGET_EDGES
                flat = self.chosen_Gmat.ravel()
                if len(flat) > TARGET_EDGES:
                    threshold = np.partition(flat, -TARGET_EDGES)[-TARGET_EDGES]
                    self.chosen_Gmat[self.chosen_Gmat < threshold] = 0.0

                # Noise Pruning
                self.chosen_Gmat[self.chosen_Gmat < DSM_PRUNE_THRESH] = 0.0

                density = np.count_nonzero(self.chosen_Gmat)
                print(f" [Optimizer] Matrix Updated. Density: {density} edges. Max Val: {np.max(self.chosen_Gmat):.2f}")

                self.walks, self.best_walk = collect_and_select_best_walks(
                    self.chosen_Gmat,
                    self.capped_node_metrics,
                    beta=WALK_BETA
                )

                self.print_dsm_basic()

                if not hasattr(self, "_node_contributions_history"):
                    self._node_contributions_history = []
                self._node_contributions_history.append(node_contributions.copy())

                return node_metrics_list, 0.0, node_contributions

            def run(self, outer_generations=OUTER_GENERATIONS, num_dsm_layers=OUTER_DSM_LAYERS):
                best_score = -np.inf

                # 1. FIX INITIALIZATION:
                # Use the random initial state as the baseline.
                # This ensures Layer 0 captures the "Jump" from noise to structure.
                baseline = self.chosen_Gmat.copy()
                dsm_decomposer = DSM_Layer_Decomposer(baseline, mode='additive')
                dsm_decomposer.current_total = baseline.copy()

                print(f"Starting Optimization: {num_dsm_layers} Layers x {outer_generations} Gens")

                # Define the "Building Blocks" for the 3 layers (based on your 12-15 node stack)
                # Layer 0: Structure (Nodes 0-5), Layer 1: Systems (Nodes 6-10), Layer 2: Skin/Ops (Nodes 11-14)
                nodes_per_layer = np.array_split(range(self.D_graph), num_dsm_layers)

                for layer_idx in range(num_dsm_layers):
                    print(f"\n>>> COMPILING LAYER {layer_idx + 1}: {['STRUCTURE', 'SYSTEMS', 'SKIN'][layer_idx]} <<<")

                    # Determine the nodes active in this specific layer
                    active_nodes = nodes_per_layer[layer_idx]

                    for gen in range(outer_generations):
                        # 1. Inner Loop (Targeting active nodes for this layer)
                        node_metrics_list = []
                        for node_idx in range(self.D_graph):
                            full_target = self.synthetic_targets[node_idx]['target']
                            D_fcm = self.candidate_dims[node_idx][0]
                            target = full_target[:D_fcm]

                            # We simulate everything, but the "Learning" is focused on the active layer
                            _, _, _, _, metrics = self.run_inner(node_idx, target, D_fcm)
                            node_metrics_list.append(metrics)

                        self.capped_node_metrics = node_metrics_list

                        # 2. Outer Loop (Topology Optimization)
                        # We pass the layer_idx to run_outer if you want to adjust the WALK_TOP_K
                        # or additive_rate per layer (e.g., higher for structure, lower for skin)
                        _, capped_score, _ = self.run_outer()

                        best_score = max(best_score, capped_score)
                        print(f" [Gen {gen+1}] Score: {capped_score:.4f}", end='\r')

                    print("")

                    # 3. SNAPSHOT: The Decomposer captures the "Delta" for this layer
                    # This is where the MUX/DEMUX logic is voucher-ed.
                    dsm_decomposer.add_snapshot(self.chosen_Gmat)

                self.dsm_layers = dsm_decomposer.layers
                print("\nOptimization Complete. All 3 Layers Compiled.")
                return best_score

            # ---------- VISUALIZATIONS & ANALYSIS ----------

            def plot_pointwise_minmax_elite(self, top_k=21):
                plt.figure(figsize=(14, 3))
                for i in range(self.D_graph):
                    dim_i = self.candidate_dims[i][0]
                    base = self.nested_reps[i][:dim_i]
                    reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)
                    y_min, y_max = reps.min(axis=0), reps.max(axis=0)
                    y_sel = base

                    y_true = self.synthetic_targets[i]['target'][:len(y_sel)]
                    if len(y_true) < len(y_sel):
                        y_true = np.pad(y_true, (0, len(y_sel) - len(y_true)), "constant")
                    else:
                        y_true = y_true[:len(y_sel)]

                    plt.subplot(1, self.D_graph, i + 1)
                    plt.fill_between(range(len(y_min)), y_min, y_max, color='skyblue', alpha=0.4, label='Elite Interval')
                    plt.plot(y_sel, 'k-', lw=2, label='Estimated')
                    plt.plot(y_true, 'r--', lw=2, label='True')
                    plt.ylim(0, 1.05)
                    plt.title(f"Node {i + 1}")
                    if i == 0: plt.legend()
                plt.tight_layout()
                plt.show()

            def plot_nested_activations(self):
                plt.figure(figsize=(12, 3))
                for i, rep in enumerate(self.nested_reps):
                    dim_i = self.candidate_dims[i][0]
                    rep_i = rep[:dim_i]
                    plt.subplot(1, self.D_graph, i + 1)
                    plt.bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
                    plt.ylim(0, 1)
                    plt.title(f"Node {i + 1}")
                plt.tight_layout()
                plt.show()

            def plot_outer_fuzzy_graph(self):
                G = nx.DiGraph()
                for i in range(self.D_graph): G.add_node(i)
                for i in range(self.D_graph):
                    for j in range(self.D_graph):
                        if i != j and abs(self.chosen_Gmat[i, j]) > 0.02:
                            G.add_edge(i, j, weight=self.chosen_Gmat[i, j])

                node_sizes = [self.best_dim_per_node[i] * 200 for i in range(self.D_graph)]
                edge_colors = ['green' if d['weight'] > 0 else 'red' for _, _, d in G.edges(data=True)]
                edge_widths = [abs(d['weight']) * 3 for _, _, d in G.edges(data=True)]

                pos = nx.spring_layout(G)
                plt.figure(figsize=(6, 6))
                nx.draw(G, pos, node_size=node_sizes, node_color='skyblue',
                        edge_color=edge_colors, width=edge_widths, arrows=True, with_labels=True)
                plt.title("Outer Fuzzy Multiplex Graph")
                plt.show()

            def print_interactions(self, return_tensor=True, verbose=True):
                D_graph = self.D_graph
                inter_dim = self.inter_layer.inter_dim
                inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

                acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
                if not acts:
                    if verbose:
                        print("No active edges above threshold.")
                    return inter_tensor if return_tensor else None

                for (i, j), vec in acts.items():
                    inter_tensor[i, j, :] = vec
                    if verbose:
                        act_str = ", ".join([f"{v:.3f}" for v in vec])
                        print(f"Node {i} -> Node {j}: [{act_str}]")
                return inter_tensor if return_tensor else None

            def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
                metrics_keys = self.MK
                D = self.D_graph
                num_metrics = len(metrics_keys)
                tensor = np.zeros((D, D, num_metrics))

                metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

                node_metrics = []
                for i, rep in enumerate(self.nested_reps):
                    metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                    node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
                node_metrics = np.array(node_metrics)

                for i in range(D):
                    for j in range(D):
                        if i == j:
                            tensor[i, j, :] = node_metrics[j]
                        else:
                            weight = np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                            tensor[i, j, :] = weight * node_metrics[j]

                if normalize:
                    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

                if verbose:
                    print("Fuzzy Metric Tensor shape:", tensor.shape)

                return tensor

            def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=None):
                if metrics_keys is None:
                    metrics_keys = self.MK
                if fuzzy_tensor is None:
                    fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

                D = self.D_graph
                num_metrics = len(metrics_keys)

                fig, axes = plt.subplots(1, num_metrics, figsize=(4 * num_metrics, 4))
                if num_metrics == 1: axes = [axes]

                im = None
                for k, key in enumerate(metrics_keys):
                    data = fuzzy_tensor[:, :, k]
                    im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
                    for i in range(D):
                        for j in range(D):
                            axes[k].text(j, i, f"{data[i, j]:.2f}", ha='center', va='center', color='white', fontsize=9)
                    axes[k].set_xticks(range(D))
                    axes[k].set_yticks(range(D))
                    axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
                    axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
                    axes[k].set_title(f'FMT - {key}')

                fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
                plt.tight_layout()
                plt.show()

            def compute_fmt_with_elite_bounds(self, top_k=21):
                metrics_keys = self.MK
                D = self.D_graph
                num_metrics = len(metrics_keys)
                tensor_bounds = np.zeros((D, D, num_metrics, 2))

                metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

                for i in range(D):
                    base = self.nested_reps[i]
                    reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)

                    metrics_matrix = np.zeros((top_k, num_metrics))
                    for idx, rep in enumerate(reps):
                        m = metrics_evaluator.compute_node_metrics(i, y=rep)
                        metrics_matrix[idx, :] = [m[k] for k in metrics_keys]

                    lower_i = metrics_matrix.min(axis=0)
                    upper_i = metrics_matrix.max(axis=0)

                    for j in range(D):
                        tensor_bounds[i, j, :, 0] = lower_i
                        tensor_bounds[i, j, :, 1] = upper_i

                return tensor_bounds

            def plot_fmt_with_bounds(self, fmt_tensor_bounds):
                D = self.D_graph
                metrics_keys = self.MK
                M_actual = len(metrics_keys)

                mean_vals = (fmt_tensor_bounds[:, :, :, 0] + fmt_tensor_bounds[:, :, :, 1]) / 2
                mean_vals = mean_vals.mean(axis=1)  # mean across targets
                mean_vals = mean_vals.mean(axis=0, keepdims=True)  # mean across nodes

                if hasattr(self, 'best_alpha') and hasattr(self, 'best_w_contrib'):
                    mean_weight = (self.best_alpha * self.best_w_contrib).mean()
                    mean_vals = mean_vals * mean_weight

                fig, ax = plt.subplots(figsize=(1.2 * M_actual + 4, 2))
                im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

                vmin, vmax = mean_vals.min(), mean_vals.max()
                for i in range(mean_vals.shape[0]):
                    for k in range(M_actual):
                        val = mean_vals[i, k]
                        color = 'white' if val < (vmin + 0.5 * (vmax - vmin)) else 'black'
                        ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

                ax.set_xticks(range(M_actual))
                ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
                ax.set_yticks([0])
                ax.set_yticklabels(['Mean across nodes'])
                ax.set_title("Weighted FMT with Bounds")
                fig.colorbar(im, ax=ax, label='Weighted Mean Metric Value')
                plt.tight_layout()
                plt.show()

            def plot_node_score_contribution(self, metrics_keys=None):
                if metrics_keys is None:
                    metrics_keys = self.MK
                D = self.D_graph
                node_contributions = np.array(self.node_score_contributions)

                if hasattr(self, 'weighted_fmt'):
                    fuzzy_tensor = np.array(self.weighted_fmt)
                else:
                    fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

                fuzzy_tensor_norm = (fuzzy_tensor - fuzzy_tensor.min()) / (fuzzy_tensor.max() - fuzzy_tensor.min() + 1e-12)
                fmt_matrix = fuzzy_tensor_norm.sum(axis=2)
                np.fill_diagonal(fmt_matrix, 0)

                raw_matrix = np.zeros((D, D))
                np.fill_diagonal(raw_matrix, node_contributions)

                total_matrix = raw_matrix + fmt_matrix

                fig, axes = plt.subplots(1, 3, figsize=(15, 4))
                matrices = [raw_matrix, fmt_matrix, total_matrix]
                titles = ["Raw Node Contribution", "Normalized FMT Contribution", "Total Contribution"]

                im = None
                for ax, mat, title in zip(axes, matrices, titles):
                    im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1)
                    for i in range(D):
                        for j in range(D):
                            ax.text(j, i, f"{mat[i, j]:.2f}", ha='center', va='center', color='white', fontsize=8)
                    ax.set_title(title)
                    ax.set_xticks(range(D))
                    ax.set_xticklabels([f"Node {i + 1}" for i in range(D)])
                    ax.set_yticks(range(D))
                    ax.set_yticklabels([f"Node {i + 1}" for i in range(D)])

                fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Contribution Value')
                plt.tight_layout()
                plt.show()

            def plot_fmt_with_run_metrics(self, metrics_keys=None):
                if metrics_keys is None:
                    metrics_keys = self.MK
                D = self.D_graph
                M_actual = len(metrics_keys)

                if not hasattr(self, 'capped_node_metrics'):
                    raise ValueError("No node metrics available. Run run_outer() first.")

                weighted_fmt = np.zeros((D, D, M_actual))
                for i in range(D):
                    for j in range(D):
                        for k, key in enumerate(metrics_keys):
                            val = self.capped_node_metrics[j][key]
                            if i != j:
                                val *= np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                            weighted_fmt[i, j, k] = val

                for i in range(D):
                    for k in range(M_actual):
                        if not METRIC_TARGET[i][k]:
                            weighted_fmt[i, :, k] = 0.0

                mean_vals = weighted_fmt.mean(axis=1)

                fig, ax = plt.subplots(figsize=(1.2 * M_actual + 4, 0.35 * D + 4))
                im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

                vmin, vmax = mean_vals.min(), mean_vals.max()
                for i in range(D):
                    for k in range(M_actual):
                        val = mean_vals[i, k]
                        color = 'white' if val < (vmin + 0.5 * (vmax - vmin)) else 'black'
                        ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

                ax.set_xticks(range(M_actual))
                ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
                ax.set_yticks(range(D))
                ax.set_yticklabels([f"Node {i}" for i in range(D)])
                ax.set_title("Weighted FMT Metrics (Actual Run Output)")
                fig.colorbar(im, ax=ax, label='Weighted Metric Value')
                plt.tight_layout()
                plt.show()

            def collect_fmt_datapoints(self):
                self.fmt_datapoints = {k: [] for k in self.MK}
                for node_idx in range(self.D_graph):
                    if len(self.PLMS[node_idx]) == 0:
                        continue
                    for weighted_input_val, metric_vals in self.PLMS[node_idx]:
                        for m, key in enumerate(self.MK):
                            if m < len(metric_vals):
                                self.fmt_datapoints[key].append((weighted_input_val, metric_vals[m]))

            def collect_metric_traces_per_node(self):
                self.metric_traces_per_node = [{} for _ in range(self.D_graph)]
                for node_idx in range(self.D_graph):
                    self.metric_traces_per_node[node_idx] = {k: [] for k in self.MK}
                    if len(self.PLMS[node_idx]) == 0:
                        continue
                    for weighted_input_val, metric_vals in self.PLMS[node_idx]:
                        for m, key in enumerate(self.MK):
                            if m < len(metric_vals):
                                self.metric_traces_per_node[node_idx][key].append((weighted_input_val, metric_vals[m]))

            def plot_fmt_per_datapoint(self, top_k=21, span=0.3, grid_size=100):
                if not hasattr(self, 'fmt_datapoints'):
                    self.collect_fmt_datapoints()

                for key, formula in zip(self.MK, METRIC_FORMULAS):
                    if key not in self.fmt_datapoints or len(self.fmt_datapoints[key]) == 0:
                        continue

                    data = np.array(self.fmt_datapoints[key])
                    x_data, y_data = data[:, 0], data[:, 1]

                    x_curve = np.linspace(x_data.min() - span, x_data.max() + span, grid_size)
                    y_curve = np.array([formula(x) for x in x_curve])

                    plt.figure(figsize=(6, 4))
                    plt.scatter(x_data, y_data, alpha=0.6, label="FMT datapoints")
                    plt.plot(x_curve, y_curve, 'r', lw=2, label="Metric equation")
                    plt.xlabel("Weighted Input")
                    plt.ylabel(f"{key} (FMT)")
                    plt.title(f"FMT per datapoint - Metric: {key}")
                    plt.legend()
                    plt.grid(alpha=0.3)
                    plt.tight_layout()
                    plt.show()

            def plot_metric_equations_per_node(self, grid_size=100, span=0.3):
                if not hasattr(self, 'metric_traces_per_node'):
                    self.collect_metric_traces_per_node()

                for node_idx in range(self.D_graph):
                    node_traces = self.metric_traces_per_node[node_idx]
                    if all(len(v) == 0 for v in node_traces.values()):
                        continue

                    plt.figure(figsize=(6, 4))
                    for key, formula in zip(self.MK, METRIC_FORMULAS):
                        if key not in node_traces or len(node_traces[key]) == 0:
                            continue
                        data = np.array(node_traces[key])
                        x_data, y_data = data[:, 0], data[:, 1]

                        x_curve = np.linspace(x_data.min() - span, x_data.max() + span, grid_size)
                        y_curve = np.array([formula(x) for x in x_curve])

                        plt.scatter(x_data, y_data, alpha=0.6, label=f"{key} datapoints")
                        plt.plot(x_curve, y_curve, 'r', lw=2, label=f"{key} equation")

                    plt.xlabel("Weighted Input")
                    plt.ylabel("Metric Value")
                    plt.title(f"Node {node_idx} - Metric Equations")
                    plt.legend()
                    plt.grid(alpha=0.3)
                    plt.tight_layout()
                    plt.show()


        # =============================================================================
        # MAIN EXECUTION BLOCK
        # =============================================================================

        if __name__ == "__main__":
            # Ensure necessary globals exist before running; otherwise this block is illustrative
            try:
                optimizer = Fuzzy_Hierarchical_Multiplex(
                    candidate_dims, D_graph,
                    synthetic_targets,
                    gamma_interlayer=0,
                    causal_flag=False
                )

                # Run Optimization
                metrics_list = optimizer.run()

                # Visualizations
            # optimizer.plot_pointwise_minmax_elite()
                #optimizer.plot_nested_activations()

                # Compute FMT with elite bounds
                #fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k + 10)

                # Plot as heatmaps
                #optimizer.plot_fmt_with_run_metrics()

                # Compute fuzzy multiplex tensor
                #fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=False)
                #optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

                # Plot Contributions & Graph
                #optimizer.plot_node_score_contribution()
                optimizer.plot_outer_fuzzy_graph()

                # Interactions
            # tensor = optimizer.print_interactions()
                #print("Tensor shape:", tensor.shape, '\n', tensor)

                # Datapoints & Equations
                #optimizer.collect_fmt_datapoints()
                #optimizer.plot_fmt_per_datapoint()
                #optimizer.collect_metric_traces_per_node()
            # optimizer.plot_metric_equations_per_node()

                # DSM Tracking Demo
                dsm_tracker = DSM_Tracker(optimizer)

                # Run extra DSM update
                optimizer.run_outer()
                dsm_tracker.update_dsms()

                # Retrieve matrices
                primary, residual = dsm_tracker.get_matrices()
                #print("Primary DSM:\n", primary)
                #rint("Residual DSM:\n", residual)

            except NameError as e:
                print(f"Error: Missing external dependency definition. \n{e}")
                print("Please ensure D_graph, DATA_MATRIX, METRIC_KEYS, etc. are defined.")

       # primary, residual = dsm_tracker.get_matrices()

        import networkx as nx
        import numpy as np

        import networkx as nx
        import numpy as np

        class TopologicalEvaluator:
            def __init__(self, dsm, node_metrics, node_types=None):
                """
                :param dsm: The adjacency matrix (np.array)
                :param node_metrics: List of dicts containing 'cost', 'quality', etc.
                :param node_types: List of 'A' or 'B' types (inferred if None)
                """
                self.dsm = np.array(dsm)
                self.metrics = node_metrics
                self.node_types = node_types if node_types else self._infer_types()
                self.G = self._build_graph()

            def _infer_types(self):
                return ["A" if m.get('quality', 0) >= m.get('cost', 0) else "B" for m in self.metrics]

            def _build_graph(self):
                G = nx.DiGraph()
                for i in range(len(self.dsm)):
                    for j in range(len(self.dsm)):
                        if self.dsm[i, j] > 0:
                            G.add_edge(i, j, weight=self.dsm[i, j])
                return G

            def calculate_system_pressure(self):
                """Calculates total pressure based on P = F/A logic across the topology."""
                total_p = 0
                edge_data = []

                for u, v in self.G.edges():
                    # Force (Potential delta)
                    v_u = np.array(list(self.metrics[u].values()))
                    v_v = np.array(list(self.metrics[v].values()))
                    force = np.linalg.norm(np.maximum(v_v - v_u, 0)) + 1e-9

                    # Area (Friction of destination)
                    area = np.mean(v_v) + 0.1

                    # Pressure
                    p = self.dsm[u, v] * (force / area)
                    total_p += p
                    edge_data.append({'link': (u, v), 'pressure': p})

                return total_p, edge_data

            def evaluate_topology(self):
                # 1. Component Analysis
                sccs = list(nx.strongly_connected_components(self.G))
                cycles = [list(c) for c in sccs if len(c) > 1]

                # 2. Hierarchical Depth (Topological Generations)
                condensed = nx.condensation(self.G)
                layers = list(nx.topological_generations(condensed))

                # 3. System Pressure
                total_p, edge_pressures = self.calculate_system_pressure()

                # 4. Criticality Score
                # High score if high depth but low cycle intensity
                fitness = (len(layers) * 10) - (len(cycles) * 5) - (total_p * 0.5)

                return {
                    "hierarchy_depth": len(layers),
                    "cycle_count": len(cycles),
                    "system_pressure": round(total_p, 4),
                    "fitness_score": round(fitness, 2),
                    "bottlenecks": sorted(edge_pressures, key=lambda x: x['pressure'], reverse=True)[:3]
                }

        import numpy as np

        # 1. GENERATE ACTUAL STATE DATA
        # We must turn the generator functions into a snapshot of data
        generators = GENERATOR_MAP
        current_state = {node: gen() for node, gen in generators.items()}

        # 2. TRANSFORM STATE INTO EVALUATOR-FRIENDLY METRICS
        # The evaluator looks for 'quality' and 'cost'. We map your architectural KPIs:
        # Quality = performance-based metrics (safety, aesthetics, completeness)
        # Cost = resource-based metrics (budget, tonnage, violations)
        evaluator_metrics = []
        METRIC_MAP = metric_feature_map
        node_list = list(METRIC_MAP.keys())

        for node in node_list:
            node_data = current_state.get(node, {})
            feat_key = METRIC_MAP[node][0]
            category = METRIC_MAP[node][1]

            val = node_data.get(feat_key, 0.5)

            # Logic: If it's a 'Prevention' or 'Alpha' category, it's Quality.
            # If it's 'Efficiency' or 'Total Cost', it relates to Cost.
            if category in ["Regulatory_Alpha", "Clinical_Safety", "Loss_Ratio_Prevention"]:
                evaluator_metrics.append({"quality": val, "cost": 0.1}) # Low baseline cost
            else:
                evaluator_metrics.append({"quality": 0.5, "cost": val / 1000.0}) # Scale budget to 0-1 range

        # 3. RUN EVALUATOR
        # Ensure 'primary' (your DSM) is indexed in the same order as node_list
        evaluator = TopologicalEvaluator(primary, evaluator_metrics)
        report = evaluator.evaluate_topology()


        # 4. OUTPUT RESULTS
                # Unify all analysis and print logic into a single command
        import numpy as np

        def run_unified_audit(primary_matrix, target_dsm, evaluator_metrics, node_list):
            """
            Unified Topological Audit comparing the 'Primary' system against the 'Target DSM'.
            """
            # 1. Setup Evaluators for both matrices
            # We are comparing Primary (The baseline/current state) vs DSM (The target/proposed state)
            eval_primary = TopologicalEvaluator(primary_matrix, evaluator_metrics)
            eval_target = TopologicalEvaluator(target_dsm, evaluator_metrics)

            # 2. Process Reports
            res_p = eval_primary.evaluate_topology()
            res_t = eval_target.evaluate_topology()

            # 3. UNIFIED OUTPUT TABLE
            print(f"\n{' TOPOLOGICAL SYSTEM AUDIT: PRIMARY VS. TARGET DSM ':=^75}")
            print(f"{'KPI Indicator':<32} | {'PRIMARY':<15} | {'TARGET DSM':<15} | {'DELTA':<10}")
            print("-" * 75)

            metrics_map = [
                ("Fitness Score (0-100)", res_p['fitness_score'], res_t['fitness_score']),
                ("System Pressure (Pa)", res_p['system_pressure'], res_t['system_pressure']),
                ("Sequential Hierarchy Depth", res_p['hierarchy_depth'], res_t['hierarchy_depth']),
                ("Coupled Feedback Cycles", res_p['cycle_count'], res_t['cycle_count'])
            ]

            for label, p_val, t_val in metrics_map:
                print(f"{label:<32} | {p_val:<15.2f} | {t_val:<15.2f} | {t_val - p_val:+.2f}")

            print("=" * 75)

            # 4. PRIMARY BOTTLENECK
            if res_p['bottlenecks']:
                p_top = res_p['bottlenecks'][0]
                p_src, p_dst = p_top['link']
                print(f"PRIMARY BOTTLENECK: {node_list[p_src]} → {node_list[p_dst]}")
                print(f"Friction Magnitude: {p_top['pressure']:.4f}")

            print("-" * 75)

            # 5. TARGET DSM BOTTLENECK
            if res_t['bottlenecks']:
                t_top = res_t['bottlenecks'][0]
                t_src, t_dst = t_top['link']
                print(f"TARGET DSM BOTTLENECK: {node_list[t_src]} → {node_list[t_dst]}")
                print(f"Friction Magnitude: {t_top['pressure']:.4f}")

        # --- EXECUTION ---
        # This compares the Primary matrix against the target DSM directly.
        run_unified_audit(primary, DSM, evaluator_metrics, node_list)

## DATA <--> METRICS <--> TP/DSM <--> GENERIC_SIMULATOR